In [5]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import pickle
import time
from dippykit.coding import huffman_encode
from dippykit.coding import huffman_decode
import zlib

import warnings
warnings.filterwarnings('ignore')

#Piotr's Retina
from utils import *

#Mani's Cythonised code
%cd cythonised_retina/scripts
import cv2
import numpy as np
from helpers import *
import classes as cy
%cd ../..

img = cv2.imread('beer.png')

/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/cythonised_retina/scripts
/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline


## Piotr's Retina

In [2]:
root = '/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/cortices/'

R = Retina(gpu=False)
R.loadLoc('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_loc.pkl')
R.loadCoeff('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_coeff.pkl')
fixation = (1080/2,1920/2)
shape = (1080, 1920)
R.prepare(shape, fixation)

C = Cortex(gpu=False)
C.loadLocs(root+'50k_Lloc_tight.pkl', root+'50k_Rloc_tight.pkl')
C.loadCoeffs(root+'50k_Lcoeff_tight.pkl', root+'50k_Rcoeff_tight.pkl')

## Mani's Cythonised Retina

In [6]:
retina_layers = 'cythonised_retina/data/retina_layers.pkl'
left_path = 'cythonised_retina/data/left_path.pkl'
right_path = 'cythonised_retina/data/right_path.pkl'

cythonised_R = cy.Retina(retina_layers)
cythonised_R.calibrate(img)

#cythonised_C = cy.Cortex(left_path, right_path)
#cythonised_C.calibrate(cythonised_R.getResult())

In [7]:
def decode_data(data):
    encoded_V = data[0]
    code_dict = data[1]
    bits = data[2]
    
    V = huffman_decode(encoded_V, symbol_code_dict=code_dict, stream_length=bits)
    return V

In [8]:
import zmq
context = zmq.Context()

print("Connecting to Client…")
client_socket = context.socket(zmq.PULL)
client_socket.bind("tcp://*:5555")

while True:
    #Determine if the data will be a regular, retinal, or cortical image
    data_type = client_socket.recv()
    
    ret, cort = False, False
    if data_type == b'Ret':
        ret = True
    elif data_type == b'Cort':
        cort = True
    
    #Whether or not to use the Cythonised Retina
    cythonised = client_socket.recv()
    
    decoding_times = []
    backprojection_times = []
   
    start = time.time()
    while True:
        data = client_socket.recv()
        if data == b'Done':
            break
        
        t1 = time.time()
        
        decompressed_data = zlib.decompress(data)
        encoded_data = pickle.loads(decompressed_data) 
        V = decode_data(encoded_data)
        
        t2 = time.time()
        
        if ret:
            if cythonised == b'True':
                V = V.reshape((3, 50001))
                cythonised_R.sampledVector = V
                cythonised_R.backProject()
                image = get_retinaBackProjected_BGR(cythonised_R)
            
            else:   
                V = V.reshape((50000, 3))
                image = R.backproject_tight(V, (1080,1920,3), fixation)
        
        elif cort:
            if cythonised == b'True':
                V = V.reshape((3, 50001))
                cythonised_R.sampledVector = V.astype('uint64')
                cythonised_C.backProject(cythonised_R.getResult())
                image = get_cortexBackProjected_BGR(cythonised_C)
            
            else:
                V = V.reshape((50000, 3))
                image = C.cort_img(V)
        
        else:
            image = V.reshape((1080, 1920, 3))
        
        t3 = time.time()
        
        decoding_times.append(t2-t1)
        backprojection_times.append(t3-t2)
        
        cv2.imshow('Image', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
    end = time.time()
    client_time = float(client_socket.recv())
    server_time = end-start
    
    cv2.destroyWindow('Image')
    cv2.waitKey(1)
    
    print("----------Report----------")
    print("Total time: %f"%(client_time+server_time))
    print("Client side total time: %f"%(client_time))
    print("Server side total time: %f"%(server_time))
    
    print("Server side decoding times: %f"%sum(decoding_times))
    print("Server side vector to image times: %f"%sum(backprojection_times))
    print("-"*30)
    print()

    

Connecting to Client…


error: Error -3 while decompressing data: incorrect header check

In [ ]:
video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
vidcap = cv2.VideoCapture(video_path)
success, img = vidcap.read()

count = 1
while success:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if count ==1:
        print(img.shape)
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    success, img = vidcap.read()
    count += 1

cv2.destroyWindow('Image')
cv2.waitKey(1)
vidcap.release()